In [25]:
# ========== ----- ========== Import Libraries ========== ----- ========== #

from joblib import dump, load
from collections import Counter
from LDP import LDP_TOP
from LDP import train
import sklearn
print(sklearn.__version__)

import numpy as np
import dlib
import cv2
import os
import re
import json
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import pickle

# ========== ----- ========== End ========== ----- ========== #

1.2.2


In [26]:
# ========== ----- ========== Load SVM Model ========== ----- ========== #

# Load the saved model from the file
linear_Deepfakes = load('models/Linear-Deepfakes.joblib')
linear_Face2Face = load('models/Linear-Face2Face.joblib')
linear_FaceSwap = load('models/Linear-FaceSwap.joblib')
linear_NeuralTextures = load('models/Linear-NeuralTextures.joblib')

# ========== ----- ========== End ========== ----- ========== #

In [ ]:
# ========== ----- ========== Reading Test Data ========== ----- ========== #

# Reading Testing Data
test_frame_folders = [
    # 'faceforensics_dataset_test/original_sequences',
    # 'faceforensics_dataset_test/manipulated_sequences/Deepfakes',
    'faceforensics_dataset_test/manipulated_sequences/Face2Face',
    'faceforensics_dataset_test/manipulated_sequences/FaceSwap',
    'faceforensics_dataset_test/manipulated_sequences/NeuralTextures'
]

# ========== ----- ========== End ========== ----- ========== #

In [ ]:
# ========== ----- ========== Feature Extraction ========== ----- ========== #

# Face detector
detector = dlib.get_frontal_face_detector()

list_of_test_LDP = []
list_of_test_labels = []

index = 0
for test_frame_folder in test_frame_folders:
    list_of_test_data = [f for f in os.listdir(
        test_frame_folder) if f.endswith('.mp4')]
    
    print(test_frame_folder, 'videos: ', len(list_of_test_data))
    for vid in list_of_test_data:
        list_of_test_LDP.append([])
        frames = []
        # Video capturing constructor.
        cap = cv2.VideoCapture(os.path.join(test_frame_folder, vid))

        # (CAP_PROP_FPS) Returns frame rate of the video (#frames / second).
        frameRate = cap.get(5)
        while cap.isOpened():  # Returns true if video capturing has been initialized already.
            # (CAP_PROP_POS_MSEC) Current position of the video file in milliseconds or video capture timestamp.
            frameId = cap.get(1)
            ret, frame = cap.read()  # The methods/functions combine VideoCapture::grab and VideoCapture::retrieve in one call. This is the most convenient method for reading video files or capturing data from decode and return the just grabbed frame. If no frames has been grabbed (camera has been disconnected, or there are no more frames in video file), the methods return false and the functions return NULL pointer.
            if ret != True:
                break

            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()

                crop_img = frame[y1:y2, x1:x2]
                if crop_img is not None and crop_img.size > 0:
                    crop_img = cv2.resize(crop_img, (128, 128))
                    gray_scale_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                    frames.append(gray_scale_img)

            # d = 3 seconds.
            if frameId % ((int(frameRate)+1)*3) == 0:
                if (len(frames) > 0):
                    frames_ldp = LDP_TOP(np.array(frames).astype(np.float64))
                    list_of_test_LDP[index].append(frames_ldp)

                frames = []
        index += 1

with open('features/Face2Face-FaceSwap-NeuralTextures-Test.pkl', 'wb') as f:
    pickle.dump(list_of_test_LDP, f)

# ========== ----- ========== End ========== ----- ========== #


In [27]:
# ========== ----- ========== Feature Loading ========== ----- ========== #

# Loading the list_of_train_LDP
with open('features/Original-Deepfakes-Test.pkl', 'rb') as f:
    test_features_p1 = pickle.load(f)

# Loading the list_of_train_labels
with open('features/Face2Face-FaceSwap-NeuralTextures-Test.pkl', 'rb') as f:
    test_features_p2 = pickle.load(f)

test_features = test_features_p1 + test_features_p2

list_of_test_LDP = test_features[:560]

# ========== ----- ========== End ========== ----- ========== #

In [28]:
# ========== ----- ========== SVM Predictions ========== ----- ========== #

linear_predictions = []

for vid in list_of_test_LDP:

    y_pred = []
    y_score = []

    df_pred = linear_Deepfakes.predict(vid)
    if np.count_nonzero(df_pred == 1) >= np.count_nonzero(df_pred == 0):
        y_score.append(np.count_nonzero(df_pred == 1))
        y_pred.append(1)
    else:
        y_score.append(np.count_nonzero(df_pred == 0))
        y_pred.append(0)

    # ============================================= #

    f2f_pred = linear_Face2Face.predict(vid)
    if np.count_nonzero(f2f_pred == 1) >= np.count_nonzero(f2f_pred == 0):
        y_score.append(np.count_nonzero(f2f_pred == 1))
        y_pred.append(1)
    else:
        y_score.append(np.count_nonzero(f2f_pred == 0))
        y_pred.append(0)

# ============================================= #

    fsw_pred = linear_FaceSwap.predict(vid)
    if np.count_nonzero(fsw_pred == 1) >= np.count_nonzero(fsw_pred == 0):
        y_score.append(np.count_nonzero(fsw_pred == 1))
        y_pred.append(1)
    else:
        y_score.append(np.count_nonzero(fsw_pred == 0))
        y_pred.append(0)

# ============================================= #

    # nt_pred = linear_NeuralTextures.predict(vid)
    # if np.count_nonzero(nt_pred == 1) >= np.count_nonzero(nt_pred == 0):
    #     y_score.append(np.count_nonzero(nt_pred == 1))
    #     y_pred.append(1)
    # else:
    #     y_score.append(np.count_nonzero(nt_pred == 0))
    #     y_pred.append(0)

# ============================================= #

    if y_pred.count(1) > 0:
        linear_predictions.append(1)
    else:
        linear_predictions.append(0)


# ========== ----- ========== End ========== ----- ========== #


In [29]:
# ========== ----- ========== SVM Accuracy ========== ----- ========== #

labels = [0]*140 + [1]*420

# calculate the accuracy score and print it out
accuracy = accuracy_score(labels, linear_predictions)
print("Linear Accuracy:", accuracy)

# calculate the confusion matrix and print it out
cm = confusion_matrix(labels, linear_predictions)
print("Linear Confusion matrix:\n", cm)

# ========== ----- ========== End ========== ----- ========== #


Linear Accuracy: 0.9125
Linear Confusion matrix:
 [[110  30]
 [ 19 401]]


In [30]:
# ========== ----- ========== Select True Classified Videos ========== ----- ========== #
indices = []
index = 0
for linear_prediction,label in zip(linear_predictions,labels):
    if linear_prediction != label :
        indices.append(index)
    
    index+=1

# ========== ----- ========== End ========== ----- ========== #

In [31]:
# ========== ----- ========== SVM Predictions ========== ----- ========== #

list_of_test_LDP = test_features[140:560]
linear_predictions = []
i = 0

for vid in list_of_test_LDP:
    if (i+140) not in indices:
        y_pred = []
        y_score = []

        df_pred = linear_Deepfakes.predict(vid)
        if np.count_nonzero(df_pred == 1) >= np.count_nonzero(df_pred == 0):
            y_score.append(np.count_nonzero(df_pred == 1))
            y_pred.append(1)
        else:
            y_score.append(np.count_nonzero(df_pred == 0))
            y_pred.append(0)

        # ============================================= #

        f2f_pred = linear_Face2Face.predict(vid)
        if np.count_nonzero(f2f_pred == 1) >= np.count_nonzero(f2f_pred == 0):
            y_score.append(np.count_nonzero(f2f_pred == 1))
            y_pred.append(1)
        else:
            y_score.append(np.count_nonzero(f2f_pred == 0))
            y_pred.append(0)

    # ============================================= #

        fsw_pred = linear_FaceSwap.predict(vid)
        if np.count_nonzero(fsw_pred == 1) >= np.count_nonzero(fsw_pred == 0):
            y_score.append(np.count_nonzero(fsw_pred == 1))
            y_pred.append(1)
        else:
            y_score.append(np.count_nonzero(fsw_pred == 0))
            y_pred.append(0)

    # ============================================= #

        # nt_pred = linear_NeuralTextures.predict(vid)
        # if np.count_nonzero(nt_pred == 1) >= np.count_nonzero(nt_pred == 0):
        #     y_score.append(np.count_nonzero(nt_pred == 1))
        #     y_pred.append(1)
        # else:
        #     y_score.append(np.count_nonzero(nt_pred == 0))
        #     y_pred.append(0)

    # ============================================= #
        
        max_score = -1
        index = 0
        target_index = -1
        for pred, score in zip(y_pred, y_score):
            if pred == 1 and max_score < score:
                max_score = score
                target_index = index
            index += 1
        linear_predictions.append(target_index+1)

    i+=1
# ========== ----- ========== End ========== ----- ========== #


In [32]:
# ========== ----- ========== Get Labels ========== ----- ========== #

df_labels = 140
f2f_labels = 140
fsw_labels = 140
# nt_labels = 140

for index in indices:
    if index >= 140 and index < 280:
        df_labels-=1

    elif index >= 280 and index < 420:
        f2f_labels -= 1

    if index >= 420 and index < 560:
        fsw_labels -= 1

    # if index >= 560 and index < 700:
        # nt_labels -= 1

# ========== ----- ========== End ========== ----- ========== #

In [33]:
# ========== ----- ========== SVM Accuracy ========== ----- ========== #


labels = [1]*df_labels + [2]*f2f_labels + [3]*fsw_labels #+ [4]*nt_labels

# calculate the accuracy score and print it out
accuracy = accuracy_score(labels, linear_predictions)
print("Linear Accuracy:", accuracy)

# calculate the confusion matrix and print it out
cm = confusion_matrix(labels, linear_predictions)
print("Linear Confusion matrix:\n", cm)

# ========== ----- ========== End ========== ----- ========== #


Linear Accuracy: 0.8478802992518704
Linear Confusion matrix:
 [[118  14   8]
 [  5 115   3]
 [  2  29 107]]


In [34]:
print(df_labels)
print(f2f_labels)
print(fsw_labels)


140
123
138
